In [1]:
!pip install git+https://github.com/seraphli/od-python.git

  Cloning https://github.com/seraphli/od-python.git to /tmp/pip-req-build-aaycg0mt
  Running setup.py bdist_wheel for od-python ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-6in4__j0/wheels/96/a0/9d/73ecc0fedd17a4cb6134556ad3c0386645657647205cc7062e
Successfully built od-python
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from __future__ import print_function
import time
import od_python
from od_python.rest import ApiException
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math
from statistics import mean, stdev
import re

In [3]:
int_ids = pd.read_csv("https://dota-match-ids.s3.amazonaws.com/international_matches.txt", 
                         header=None, names=["match_id"])

In [4]:
int_ids.head()

,match_id
0,4986461644
1,4986362254
2,4986260666
3,4986133311
4,4985891168


In [5]:
prem_ids = pd.read_csv("https://dota-match-ids.s3.amazonaws.com/premier_matches.txt", 
                         header=None, names=["match_id"])

In [6]:
prem_ids.head()

,match_id
0,4986461644
1,4986362254
2,4986260666
3,4986133311
4,4985891168


In [7]:
pro_ids = pd.read_csv("https://dota-match-ids.s3.amazonaws.com/pro_matches.txt", 
                         header=None, names=["match_id"])

In [8]:
pro_ids.head()

,match_id
0,5108885702
1,5108800438
2,5108735167
3,5108711156
4,5108703089


In [9]:
def get_pro_matches(match_list):
    m_ids = []
    durations = []
    s_times = []
    r_t_id = []
    r_name = []
    d_t_id = []
    d_name = []
    l_id = []
    l_name = []
    s_id = []
    s_type = []
    r_score = []
    d_score = []
    r_win = []

    pro_matches_api = od_python.ProMatchesApi()

    # Highest Match_ID as of 11/11/2019 - 5108977506
    match_id = 5108977506
    counter = 0

    while match_id > 17955123:
        try:
            if counter > 40:
                time.sleep(90)
                counter = 0
                print(match_id)
            else:
                api_response = pro_matches_api.pro_matches_get(less_than_match_id=match_id)
                counter = counter + 1
                for entry in api_response:
                    if entry.match_id in set(match_list):
                        m_ids.append(entry.match_id)
                        durations.append(entry.duration)
                        s_times.append(entry.start_time)
                        r_t_id.append(entry.radiant_team_id)
                        r_name.append(entry.radiant_name)
                        d_t_id.append(entry.dire_team_id)
                        d_name.append(entry.dire_name)
                        l_id.append(entry.leagueid)
                        l_name.append(entry.league_name)
                        s_id.append(entry.series_id)
                        s_type.append(entry.series_type)
                        r_score.append(entry.radiant_score)
                        d_score.append(entry.dire_score)
                        r_win.append(entry.radiant_win)
                    if entry.match_id < match_id:
                        match_id = entry.match_id
        except ApiException as e:
            print("Exception when calling LeaguesApi->leagues_get: %s\n" % e)
            match_id = 0
               
    data_dict = {
        "match_id": m_ids, 'duration': durations, "start_time": s_times, "radiant_team_id": r_t_id,
        "radiant_name": r_name, "dire_team_id": d_t_id, "dire_name": d_name, "leagueid": l_id,
        "league_name": l_name, "series_id": s_id, "series_type": s_type, "radiant_score": r_score,
        "dire_score": d_score, "radiant_win": r_win
    }
    
    return data_dict

In [10]:
international_promatch_data = get_pro_matches(int_ids["match_id"])

4998064081
4766080271
4486993446
4212332727
3851000926
3567692495
3252995512
2975314149
2670502591
2370369922
2035002016
1689526977
1376354000
1100147939
891604751
683504655
426688355
193154007


In [11]:
df_int = pd.DataFrame(data=international_promatch_data)
df_int.head()

,match_id,duration,start_time,radiant_team_id,radiant_name,dire_team_id,dire_name,leagueid,league_name,series_id,series_type,radiant_score,dire_score,radiant_win
0,4986461644,1474,1566733817,2586976.0,OG,2163.0,Team Liquid,10749,The International 2019,360940.0,2.0,20,11,True
1,4986362254,1403,1566730459,2163.0,Team Liquid,2586976.0,OG,10749,The International 2019,360940.0,2.0,11,36,False
2,4986260666,1945,1566726706,2163.0,Team Liquid,2586976.0,OG,10749,The International 2019,360940.0,2.0,11,40,False
3,4986133311,2914,1566721992,2163.0,Team Liquid,2586976.0,OG,10749,The International 2019,360940.0,2.0,33,24,True
4,4985891168,2194,1566713543,15.0,PSG.LGD,2163.0,Team Liquid,10749,The International 2019,360881.0,1.0,8,30,False


In [12]:
df_int.to_csv("international_promatch_data.csv")

# --------------------------------------------

In [ ]:
prem_promatch_data = get_pro_matches(prem_ids["match_id"])

4998064081
4766080271


In [ ]:
df_prem = pd.DataFrame(data=prem_promatch_data)
df_prem.head()

In [ ]:
df_prem.to_csv("premier_promatch_data.csv")

# --------------------------------------------

In [ ]:
prof_promatch_data = get_pro_matches(pro_ids["match_id"])

In [ ]:
df_prof = pd.DataFrame(data=prof_promatch_data)
df_prof.head()

In [ ]:
df_prof.to_csv("professional_promatch_data.csv")